# Universidad del Valle de Guatemala
## Security Data Science - 10
* Jose Abraham Gutierrez Corado - 19111
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 2: Detección de SPAM

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline

### Explorar Datos

In [17]:
df1 = pd.read_csv('completeSpamAssassin.csv')
df1.drop(["Unnamed: 0"], axis=1, inplace=True)
df1.sample(5)

,Body,Label
2906,">>>>> On Fri, 13 Sep 2002, ""Tony"" == Tony Nuge...",0
5209,">\n>\n> The question then becomes, why is know...",0
3293,"Folks, Some of you seem to have hardcoded hono...",0
4942,\nThis is a stable beta release. Please to up...,0
392,NEED Health Insurance? \n In addition to featu...,1


In [18]:
df2 = pd.read_csv('enronSpamSubset.csv')
df2.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)
df2.sample(5)

,Body,Label
3390,Subject: soft at incredibly low prices\n any s...,1
8557,Subject: vince and vasant :\n here is a brief ...,0
4684,Subject: interested in an invitation\n * this ...,1
3200,Subject: officiai information from hsbc bank p...,1
5625,Subject: power trading\n hi folks :\n very gla...,0


### Merge de los df's

In [19]:
dfNew = df1.merge(df2, how="right")
dfNew.sample(5)

,Body,Label
6390,Subject: mobil beaumont - marol\n just want to...,0
7382,"Subject: planned pre prc meetings\n sally ,\n ...",0
1423,Subject: shop for holiday gifts . free shippin...,1
7371,Subject: re : onondaga\n louise :\n i ' d like...,0
1737,"Subject: repelled , america ' s growth stocks\...",1


### Pre Procesamiento

In [ ]:
#Conversión de Mayusculas a Minusculas
def Conv_Up_to_Low(df):
    sentences = []
    for sentence in df.Body:
        sentences.append(sentence.lower)
        
    df["Body"] = sentences
    
    return df

#Acentos
import unicodedata
def acccents(df):
    sentences = []
    for sentence in df.Body:
        sentences.append(unicodedata.normalize('NFKD', sentence).encode('ascii','ignore').decode('utf-8','ignore'))
    
    df["Body"] = sentences
    
    return df

#Caracteres especiales, números y simbolos
import re
def removeSpecialCharacters(df):
    patron = r'[^a-zA-Z0-9\s]'
    sentences = []
    for sentence in df.Body:
        if patron in sentence:
            sentences.append(sub(patron,'', sentence))
        else:
            sentences.append(sub(r'[^a-zA-Z\s]', '', sentence))
    
    df["Body"] = sentences
    
#Contracciones en el texto
import contractions
def contrac(df):
    list(contractions.contractions_dict.items())[:10]
    sentences = []
    for sentence in df.Body:
        sentences.append(contractions.fix(sentence))
        
    df["Body"] = sentences

#Stop Words
import nltk
nltk.download('stopwords')

def Stop_word(df):
    for sentence in df.Body:
        tokens = nltk.word_tokenize(sentence.lower())
        stop_words_spanish = nltk.corpus.stopwords.words('spanish')
        tokens_filtrados = [token for token in tokens if token not in stop_words_spanish]
        stop_words_english = nltk.corpus.stopwords.words('english')
        tokens_english = nltk.word_tokenize(contractions.fix(sentence))
        tokens_filtrados_english = [token for token in tokens_english if token not in stop_words_english]
        sentences.append(tokens_filtrados_english)
        


#Lemmatization